####  ①搭建环境

In [ ]:
import os
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

print(torch.__version__)

#### ②创建模型

In [ ]:
net = nn.Sequential()
net.add_module( 'C1',nn.Conv2d(1, 6, 5) ) #in_channels=1, out_channels=6, kernel_size=5
net.add_module( 'C1-activation', nn.Sigmoid() )
net.add_module( 'S2', nn.MaxPool2d(2, 2) ) #kernel_size=2, stride=2
net.add_module( 'C3', nn.Conv2d(6, 16, 5)  )
net.add_module( 'C3-activation', nn.Sigmoid() )
net.add_module( 'S4', nn.MaxPool2d(2, 2) ) #kernel_size=2, stride=2
net.add_module( 'S4-flatten', nn.Flatten() )
net.add_module( 'C5', nn.Linear(16*4*4, 120) )
net.add_module( 'C5-activation', nn.Sigmoid() )
net.add_module( 'F6', nn.Linear(120, 84) )
net.add_module( 'F6-activation', nn.Sigmoid() )
net.add_module( 'OUTPUT', nn.Linear(84, 10) )

In [ ]:

print(net)

In [ ]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)
list(train_iter)[0][0][0][0][0]
X = list(train_iter)[0][0]
#len(list(train_iter)[0][0][0][0][0])
y_hat = net(X)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = d2l.get_current_device()
print(device)

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [ ]:
lr, num_epochs = 0.9, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
start = time.time()
#d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
print(time.time() - start)